## Dense Passage Retrieval: Retrieve Top K matching passages 

In [ ]:
%%capture

!pip install cohere-sagemaker

#### Imports 

In [2]:
from cohere_sagemaker import CohereError
from cohere_sagemaker import Client
from requests.auth import HTTPBasicAuth
import requests
import logging 
import boto3
import yaml
import json
import os

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [4]:
logger.info(f'Using requests=={requests.__version__}')
logger.info(f'Using pyyaml=={yaml.__version__}')

Using requests==2.28.2
Using pyyaml==6.0


#### Setup essentials

In [5]:
TEXT_EMBEDDING_MODEL_ENDPOINT_NAME = 'huggingface-textembedding-gpt-j-6b-fp16-1680825746'
TEXT_GENERATION_MODEL_ENDPOINT_NAME = 'cohere-medium-1680827379'

CHUNKS_DIR_PATH = './data/chunks'
sagemaker_client = boto3.client('runtime.sagemaker')
cohere_client = Client(endpoint_name=TEXT_GENERATION_MODEL_ENDPOINT_NAME)

In [6]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

es_username = config['credentials']['username']
es_password = config['credentials']['password']

domain_endpoint = config['domain']['endpoint']
domain_index = config['domain']['index']

In [7]:
URL = f'{domain_endpoint}/{domain_index}/_search'
logger.info(f'URL for Elasticsearch index = {URL}')

URL for Elasticsearch index = https://search-semantic-search-hryn56c5jy43yryimohz4ajvyi.us-east-1.es.amazonaws.com/legal-passages/_search


Refer to https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html for more info.

#### Encode question using SageMaker JumpStart's text embedding model endpoint

In [8]:
prompt = 'What is the definition of crime of battery?'

In [9]:
payload = {'text_inputs': [prompt]}
payload = json.dumps(payload).encode('utf-8')
response = sagemaker_client.invoke_endpoint(EndpointName=TEXT_EMBEDDING_MODEL_ENDPOINT_NAME, 
                                            ContentType='application/json', 
                                            Body=payload)
body = json.loads(response['Body'].read())
embedding = body['embedding'][0]

#### Find top k (k=3) matching passages aligned in context to the encoded question

In [10]:
K = 5

In [11]:
query = {
    'size': K,
    'query': {
        'knn': {
          'embedding': {
            'vector': embedding,
            'k': K
          }
        }
      }
    }

In [12]:
response = requests.post(URL, auth=HTTPBasicAuth(es_username, es_password), json=query)
response_json = response.json()
hits = response_json['hits']['hits']

#### Generate answers using SageMaker JumpStart's text generation model by leveraging the previously matched passages 

In [13]:
for hit in hits:
    score = hit['_score']
    passage = hit['_source']['passage']
    doc_id = hit['_source']['doc_id']
    passage_id = hit['_source']['passage_id']
    qa_prompt = f'Context={passage}\nQuestion={prompt}\nAnswer='
    
    response = cohere_client.generate(prompt=prompt, 
                                      max_tokens=512, 
                                      temperature=0.25, 
                                      return_likelihoods='GENERATION')
    
    answer = response.generations[0].text.strip().replace('\n', '')
    logger.info(f'Answer:\n{answer}')
    logger.info(f'Reference:\nDocument = {doc_id} | Passage = {passage_id} | Score = {score}')
    
if not hits:
    logger.warn('No matching documents found!')

Answer:
The crime of battery is committed when a person intentionally touches someone else in a way that causes injury.
Reference:
Document = 005 | Passage = 45 | Score = 0.64828235
Answer:
The crime of battery is committed when a person intentionally touches another person in a harmful or offensive way.
Reference:
Document = 005 | Passage = 44 | Score = 0.6335997
Answer:
The crime of battery is defined as "an unlawful and intentional application of force upon another person."
Reference:
Document = 005 | Passage = 7 | Score = 0.6297361
Answer:
The crime of battery is committed when a person intentionally touches another person against their will.
Reference:
Document = 005 | Passage = 74 | Score = 0.6294636
Answer:
The crime of battery is defined as "willfully and maliciously causing harm to another person."
Reference:
Document = 007 | Passage = 12 | Score = 0.6268758
